In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [11]:
from sklearn.metrics import mean_squared_error
from prophet import Prophet
from flask import Flask, request
from flask_cors import CORS
import pandas as pd
import datetime
import statsmodels.api as sm
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

In [32]:
def arima_AIC(data, p = 4, d = 4, q = 4):

    # MSE
    period = 1
    L = len(data)
    train = data[ : (L - period)]
    test = data[ - period: ]
    mse_r = []
    # AIC
    best_pdq =["AIC_pdq", 10000, "MSE", 10000]
    AIC = []
    
    for i in range(p): # AR
        
        for j in range(1, d): # I
            
            for k in range(q): # MA
                
                model = sm.tsa.arima.ARIMA(data, order = (i,j,k))
                fitted = model.fit()
                # MSE
                forecast = fitted.forecast(step = period, alpha = 0.05)
                mse = mean_squared_error(test, forecast)
                mse_r.append(mse)
                # AIC
                AIC.append(fitted.aic)
                
                if fitted.aic < best_pdq[1]:
                    
                    best_pdq[0] = (i, j, k)
                    best_pdq[1] = fitted.aic
                    
                if mse < best_pdq[1]:
                    
                    best_pdq[2] = (i, j, k)
                    best_pdq[3] = mse
    
    return best_pdq[0], best_pdq[2]

def arima_predict(full_data, best_pdq_MSE):

    model = sm.tsa.ARIMA(full_data['Open'], order = best_pdq_MSE)
    fitted = model.fit()
    number_of_steps = 10
    forecast = fitted.forecast(steps = number_of_steps)

    ten_days_after = forecast.values[9]
    one_days_after = forecast.values[0]

    percentage_change = ((ten_days_after - one_days_after) / one_days_after) * 100

    return forecast, percentage_change

def prophet_predict(full_data):
    
    df = pd.DataFrame(full_data['Open'])
    df.reset_index(inplace = True)
    df = df.rename(columns = {'Date': 'ds', 'Open': 'y'})
    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=10)
    forecast = model.predict(future)
    predicted = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10).reset_index()
    ten_days_after = predicted['yhat'].iloc[9]
    one_days_after = predicted['yhat'].iloc[0]
    percentage_change = ((ten_days_after - one_days_after) / one_days_after) * 100
    
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], percentage_change


In [35]:
full_data_one = yf.download("BTC-USD", start = "2023-01-01", end = datetime.date.today().strftime("%Y-%m-%d"), interval="5d")
full_data_one

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-01,16547.914062,16630.439453,16521.234375,16625.080078,16625.080078,9244361700
2023-01-06,16836.472656,16991.994141,16716.421875,16951.968750,16951.968750,14413662913
2023-01-11,17446.359375,17934.896484,17337.994141,17934.896484,17934.896484,18372283782
2023-01-16,20882.224609,21360.875000,20715.746094,21169.632812,21169.632812,26792494050
2023-01-21,22677.427734,23282.347656,22511.833984,22777.625000,22777.625000,32442278429
...,...,...,...,...,...,...
2024-01-26,39936.816406,42209.386719,39825.691406,41816.871094,41816.871094,25598119893
2024-01-31,42946.250000,43717.406250,42298.945312,42582.605469,42582.605469,24673628793
2024-02-05,42577.621094,43494.250000,42264.816406,42658.667969,42658.667969,18715487317


In [39]:
best_pdq_AIC_one, best_pdq_MSE_one = arima_AIC(full_data_one['Open'], 4, 4, 4)
print(best_pdq_AIC_one)
print(best_pdq_MSE_one)

(1, 3, 3)
(0, 1, 0)


In [38]:
arima_predict(full_data_one, best_pdq_MSE_one)

(2024-02-20    51836.785156
 2024-02-25    51836.785156
 2024-03-01    51836.785156
 2024-03-06    51836.785156
 2024-03-11    51836.785156
 2024-03-16    51836.785156
 2024-03-21    51836.785156
 2024-03-26    51836.785156
 2024-03-31    51836.785156
 2024-04-05    51836.785156
 Freq: 5D, Name: predicted_mean, dtype: float64,
 0.0)

In [37]:
arima_forecast_one, percentage_change_one = arima_predict(full_data_one, best_pdq_MSE_one)
arima_forecast_one

2024-02-20    51836.785156
2024-02-25    51836.785156
2024-03-01    51836.785156
2024-03-06    51836.785156
2024-03-11    51836.785156
2024-03-16    51836.785156
2024-03-21    51836.785156
2024-03-26    51836.785156
2024-03-31    51836.785156
2024-04-05    51836.785156
Freq: 5D, Name: predicted_mean, dtype: float64

In [34]:
prophet_forecast_one, p_change_one = prophet_predict(full_data_one)
prophet_forecast_one

12:48:13 - cmdstanpy - INFO - Chain [1] start processing
12:48:13 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper
0,2023-01-01,20207.467158,17278.762601,23278.150393
1,2023-01-06,20009.209767,16765.051382,22938.269124
2,2023-01-11,20688.816051,17952.357771,23481.217450
3,2023-01-16,21329.999213,18473.057492,24253.290773
4,2023-01-21,22062.311079,18957.517322,25065.019160
...,...,...,...,...
88,2024-02-21,47854.324998,44832.366646,50575.534666
89,2024-02-22,49364.751789,46380.224649,52381.240855
90,2024-02-23,47724.091954,44883.611810,50757.421533
91,2024-02-24,49051.242815,46245.770672,52041.292792
